In [ ]:
    nb_final_model_nbfinal = nb_finalLM(y_nbfinal, X_nbfinal, groups)
        try:
            nb_final_results_nbfinal = nb_final_model_nbfinal.fit()
            nb_final_summary_nbfinal = str(nb_final_results_nbfinal.summary())
            print("\nApproximated Multilevel Model Results With nbfinal Transformations:")
            print(nb_final_summary_nbfinal)
            
            # Add to document
            doc.add_paragraph('Model Summary With nbfinal Transformations:')
            summary_paragraph_nbfinal = doc.add_paragraph()
            summary_run_nbfinal = summary_paragraph_nbfinal.add_run(nb_final_summary_nbfinal)
            summary_run_nbfinal.font.name = 'Courier New'  # Use monospace font
            #summary_run.font.size = Pt(10)  # Optional: Adjust font size
            #for line in nb_final_summary.split('\n'):
                #doc.add_paragraph(line)
            
            # Add variance components
            doc.add_paragraph('\nVariance Components With nbfinal Transformations:')
            vc_table = doc.add_table(rows=3, cols=2)
            vc_table.style = 'Table Grid'
            vc_table.cell(0, 0).text = 'Component'
            vc_table.cell(0, 1).text = 'Estimate'
            vc_table.cell(1, 0).text = 'State Random Effect Variance'
            vc_table.cell(1, 1).text = f"{nb_final_results_nbfinal.cov_re.iloc[0, 0]:.4f}"
            vc_table.cell(2, 0).text = 'Residual Variance'
            vc_table.cell(2, 1).text = f"{nb_final_results_nbfinal.scale:.4f}"
            
            # Calculate intraclass correlation coefficient (ICC)
            state_var_nbfinal = nb_final_results_nbfinal.cov_re.iloc[0, 0]
            residual_var_nbfinal = nb_final_results_nbfinal.scale
            icc_nbfinal = state_var_nbfinal / (state_var_nbfinal + residual_var_nbfinal)

            # Add model summary to document
            #doc.add_paragraph('\nnb_final Summary:')
            #for line in nb_final_summary.split('\n'):
                #doc.add_paragraph(line)
            
            # Convert coefficients to incident rate ratios (IRR)
            print("\nIncident Rate Ratios (IRR) nb_final Model With nbfinal Transformations:")
            irr_nb_final_nbfinal = np.exp(nb_final_results_nbfinal.params)
            irr_conf_nb_final_nbfinal = np.exp(nb_final_results_nbfinal.conf_int())
            irr_df_nb_final_nbfinal = pd.DataFrame({'IRR': irr_nb_final_nbfinal, 'Lower CI': irr_conf_nb_final_nbfinal[0], 'Upper CI': irr_conf_nb_final_nbfinal[1], 
                                'P-value': nb_final_results_nbfinal.pvalues})
            print(irr_df_nb_final_nbfinal)
            
            # Add IRR table to document
            doc.add_paragraph('\n')
            doc.add_heading('Incident Rate Ratios (IRR) nb_final Model With nbfinal Transformations', level=2)
            irr_table_nb_final = doc.add_table(rows=len(irr_df_nb_final_nbfinal)+1, cols=5)
            irr_table_nb_final.style = 'Table Grid'
            irr_table_nb_final.cell(0, 0).text = 'Variable'
            irr_table_nb_final.cell(0, 1).text = 'IRR'
            irr_table_nb_final.cell(0, 2).text = 'Lower CI'
            irr_table_nb_final.cell(0, 3).text = 'Upper CI'
            irr_table_nb_final.cell(0, 4).text = 'P-value'
            
            for i, (var, row) in enumerate(irr_df_nb_final_nbfinal.iterrows(), 1):
                irr_table_nb_final.cell(i, 0).text = str(var)
                irr_table_nb_final.cell(i, 1).text = f"{row['IRR']:.4f}"
                irr_table_nb_final.cell(i, 2).text = f"{row['Lower CI']:.4f}"
                irr_table_nb_final.cell(i, 3).text = f"{row['Upper CI']:.4f}"
                irr_table_nb_final.cell(i, 4).text = f"{row['P-value']:.4f}"
            
            doc.add_paragraph(f'\nIntraclass Correlation Coefficient (ICC): {icc:.4f}')
            doc.add_paragraph('The ICC represents the proportion of the total variance in length of stay ' +
                             'that is attributable to differences between states.')
            # After fitting the nb_final model
            # Compute residuals and fitted values
            df_clean_model['fitted_nbfinal'] = nb_final_results_nbfinal.fittedvalues
            df_clean_model['residuals_nbfinal'] = nb_final_results_nbfinal.resid

            # Plot residuals vs fitted values
            plt.figure(figsize=(10, 6))
            plt.scatter(df_clean_model['fitted_nbfinal'], df_clean_model['residuals_nbfinal'], alpha=0.5)
            plt.axhline(y=0, color='r', linestyle='-')
            plt.xlabel('Fitted Values')
            plt.ylabel('Residuals')
            plt.title('Residuals vs Fitted Values')
            plt.tight_layout()
            residuals_vs_fitted_img_nbfinal = BytesIO()
            plt.savefig(residuals_vs_fitted_img_nbfinal, format='png')
            residuals_vs_fitted_img_nbfinal.seek(0)
            plt.close()

            # Add to document
            doc.add_paragraph('\n')
            doc.add_heading('Post-Estimation Diagnostics', level=2)
            doc.add_picture(residuals_vs_fitted_img_nbfinal, width=Inches(6))
            doc.add_paragraph('Figure 8: Residuals vs Fitted Values With nbfinal Transformations')

            # Q-Q plot for normality
            plt.figure(figsize=(10, 6))
            stats.probplot(df_clean_model['residuals_nbfinal'], dist="norm", plot=plt)
            plt.title('Q-Q Plot of Residuals With nbfinal Transformations')
            plt.tight_layout()
            qq_plot_img_nbfinal = BytesIO()
            plt.savefig(qq_plot_img_nbfinal, format='png')
            qq_plot_img_nbfinal.seek(0)
            plt.close()

            # Add to document
            doc.add_picture(qq_plot_img_nbfinal, width=Inches(6))
            doc.add_paragraph('Figure 9: Q-Q Plot of Residuals With nbfinal Transformations')

            # Fit a nbfinal linear model (no random effects)
            ols_model = smf.ols(formula, df_clean_nb)
            ols_results = ols_model.fit()

            # Compute the likelihood ratio test
            lr_stat_nbfinal = -2 * (ols_results.llf - nb_final_results_nbfinal.llf)
            p_value_nbfinal = stats.chi2.sf(lr_stat_nbfinal, df=1)  # df=1 for one random effect
            doc.add_paragraph(f'\nLikelihood Ratio Test for Random Effects with nbfinal Model: Statistic = {lr_stat_nbfinal:.2f}, P-value = {p_value_nbfinal:.4f}')


            # Pseudo-R² (McFadden's R² approximation)
            null_model_nbfinal = smf.nb_finallm("nbfinal_los_capped ~ 1", df_clean_model, groups=df_clean_model['us_state_enc'])
            null_results_nbfinal = null_model_nbfinal.fit()
            pseudo_r2_nbfinal = 1 - (nb_final_results_nbfinal.llf / null_results_nbfinal.llf)
            doc.add_paragraph(f'\nPseudo-R² (McFadden) With nbfinal Transformations: {pseudo_r2_nbfinal:.4f}')

            

            # Check VIF for continuous variables
            from statsmodels.stats.outliers_influence import variance_inflation_factor
            X_continuous_nbfinal = X_nbfinal[[col for col in X_nbfinal.columns if col != 'const']]  # Exclude intercept
            vif_data_nbfinal = pd.DataFrame()
            vif_data_nbfinal["Variable"] = X_continuous_nbfinal.columns
            vif_data_nbfinal["VIF"] = [variance_inflation_factor(X_continuous_nbfinal.values, i) for i in range(X_continuous_nbfinal.shape[1])]
            print("VIF for continuous and dummy variables With nbfinal Transformations:")
            print(vif_data_nbfinal)
            doc.add_paragraph('\nVariance Inflation Factor (VIF) for Continuous and Categorical Variables With nbfinal Transformations:')
            vif_table = doc.add_table(rows=len(vif_data_nbfinal)+1, cols=2)
            vif_table.style = 'Table Grid'
            vif_table.cell(0, 0).text = 'Variable'
            vif_table.cell(0, 1).text = 'VIF'
            for i, (var, vif) in enumerate(zip(vif_data_nbfinal["Variable"], vif_data_nbfinal["VIF"]), 1):
                vif_table.cell(i, 0).text = str(var)
                vif_table.cell(i, 1).text = f"{vif:.4f}"
            doc.add_paragraph('VIF values above 10 indicate potential multicollinearity issues.')
        except ValueError as ve:
            error_msg = f"ValueError in nb_final model fitting: {str(ve)}"
            print(error_msg)
            doc.add_paragraph(error_msg)
            doc.add_paragraph("The nb_final effects model failed to converge. This can happen due to " +
                             "insufficient variation in the grouping variable or other model specification issues.")
        except RuntimeError as re:
            error_msg = f"RuntimeError in nb_final model fitting: {str(re)}"
            print(error_msg)
            doc.add_paragraph(error_msg)
            doc.add_paragraph("The nb_final effects model failed to converge. This can happen due to " +
                             "insufficient variation in the grouping variable or other model specification issues.")    


        except Exception as e:
            error_msg = f"Error fitting nb_final model: {str(e)}"
            print(error_msg)
            doc.add_paragraph(error_msg)
            doc.add_paragraph("The nb_final effects model failed to converge. This can happen due to " +
                             "insufficient variation in the grouping variable or other model specification issues.")
